# Pull transcripts of S&P500 Companies (Python)
Author: WRDS

Description: How to retrieve transcripts of S&P 500 companies using CRSP, CCM, and CIQ Transcripts

## Pull Transcripts of S&P 500 constituents in 2015
Import packages and connect to WRDS server

In [ ]:
import pandas as pd
import wrds

db = wrds.Connection()

Get S&P 500 Companies in 2015 from CRSP

In [ ]:
# S&P 500 Constituents in the year 2015 from CRSP
# Merge with GVKEY using CCM Linktable
sql_query = '''
            SELECT a.*, b.gvkey, b.liid, b.linkdt, b.linkenddt
            FROM (
                SELECT * 
                FROM crsp.dsp500list
                WHERE start <= make_date(2015, 1, 1)
                  AND ending >= make_date(2015, 12, 31)
            ) AS a
            LEFT JOIN (
                SELECT * 
                FROM crsp.ccmxpf_lnkhist
                WHERE linkdt <= make_date(2015, 1, 1)
                  AND (linkenddt >= make_date(2015, 12, 31) OR linkenddt IS NULL)
            ) AS b
            ON a.permno = b.lpermno
            AND b.linktype IN ('LU', 'LC')
            AND b.linkprim IN ('P', 'C');
            '''

snp500_crsp_gvkey = db.raw_sql(sql_query)

Pull Transcripts data with full-text from 2015

Sample selection:

- Transcripts of earnings conference calls
    - Key Development Event Type ID `keydeveventtypeid` = 48
- The final copy of each transcript that is edited, proofed, or audited
    - Final copy: `transcriptpresentationtypeid` = 5.0

Merge GVKEY with Transcripts data

In [ ]:
sql_query = '''
            SELECT a.*,
                   b.symbolvalue AS gvkey,
                   c.*, 
                   d.componenttext
            FROM (
                SELECT *
                FROM ciq.wrds_transcript_detail 
                WHERE keydeveventtypeid = 48 
                  AND transcriptpresentationtypeid = 5 
                  AND date_part('year', mostimportantdateutc) = 2015
            ) AS a
            LEFT JOIN (
                SELECT *
                FROM ciq.wrds_ciqsymbol 
                WHERE symboltypecat = 'gvkey'
            ) AS b
              ON a.companyid = b.companyid
            LEFT JOIN ciq.wrds_transcript_person AS c 
              ON a.transcriptid = c.transcriptid
            LEFT JOIN ciq.ciqtranscriptcomponent AS d 
              ON c.transcriptid = d.transcriptid 
             AND c.transcriptcomponentid = d.transcriptcomponentid
            ORDER BY a.transcriptid, c.transcriptcomponentid, a.companyid;
            '''

tr_detail_gvkey = db.raw_sql(sql_query)

Merge transcripts and `permno` via `gvkey`

In [ ]:
# Obtain transcripts data for GVKEYs in the S&P500 Contituent list
snp500_transcripts = tr_detail_gvkey[tr_detail_gvkey.gvkey.isin(snp500_crsp_gvkey.gvkey.tolist())]

# Remove observations with missing GVKEY
snp500_transcripts = snp500_transcripts[pd.notna(snp500_transcripts.gvkey)]

## To avoid excessive memory usage: Use SQL Query as much as possible
Using sql commands as much as possible can be a basic solution for excessive memory usage issues. The code below is an example of querying S&P 500 transcripts using only SQL syntax instead of loading all tables into memory and working with python `pandas`.

The code does:

1. Retrieving analysts' questions in S&P 500 companies' earnings conference calls in December 2015
    1. Analyst: `speakertypeid` = 3
    2. Questions: `transcriptcomponenttypeid` = 3
    3. Earnings conference call: `keydeveventtypeid` = 48
2. Only using original CIQ tables (**Transcripts, Key Developments, Common**)
3. Keeping all copies of the transcripts

In [ ]:
sql_query = '''
             WITH snp500_query AS(
                SELECT a.*, 
                       b.gvkey, 
                       b.liid, 
                       b.linkdt, 
                       b.linkenddt
                FROM (
                    SELECT * 
                    FROM crsp.dsp500list
                    WHERE start <= make_date(2015, 1, 1)
                      AND ending >= make_date(2015, 12, 31)
                ) AS a
                LEFT JOIN (
                    SELECT * 
                    FROM crsp.ccmxpf_lnkhist
                    WHERE linkdt <= make_date(2015, 1, 1)
                      AND (linkenddt >= make_date(2015, 12, 31) OR linkenddt IS NULL)
                ) AS b 
                  ON a.permno = b.lpermno
                AND b.linktype IN ('LU', 'LC')
                AND b.linkprim IN ('P', 'C')
             )

             SELECT b.transcriptcreationdateutc,
                    f.companyid,
                    f.companyname,
                    b.keydevid,
                    i.headline,
                    i.mostimportantdateutc,
                    a.transcriptid,
                    b.transcriptcollectiontypeid,
                    b.transcriptpresentationtypeid,
                    a.transcriptcomponentid,
                    a.componentorder,
                    a.transcriptcomponenttypeid,
                    h.transcriptcomponenttypename,
                    e.transcriptpersonname,
                    e.companyname AS speaker_companyname,
                    e.speakertypeid,
                    g.speakertypename,
                    a.componenttext
             FROM ciq_transcripts.ciqtranscriptcomponent AS a
             JOIN ciq_transcripts.ciqtranscript AS b
               ON a.transcriptid = b.transcriptid
             JOIN ciq_keydev.ciqkeydevtoobjecttoeventtype AS d
               ON b.keydevid = d.keydevid
             JOIN ciq_transcripts.ciqtranscriptperson AS e
               ON a.transcriptpersonid = e.transcriptpersonid
             JOIN ciq_transcripts.ciqtranscriptspeakertype AS g
               ON e.speakertypeid = g.speakertypeid
             JOIN ciq_common.ciqcompany AS f
               ON d.objectid = f.companyid
             JOIN ciq_transcripts.ciqtranscriptcomponenttype AS h
               ON a.transcriptcomponenttypeid = h.transcriptcomponenttypeid
             JOIN ciq_keydev.ciqkeydev AS i
               ON b.keydevid = i.keydevid
             JOIN ciq_common.wrds_gvkey AS k
               ON f.companyid = k.companyid
             JOIN snp500_query AS j
               ON k.gvkey = j.gvkey
             WHERE date_part('year', i.mostimportantdateutc) = 2015
               AND date_part('month', i.mostimportantdateutc) = 12
               AND a.transcriptcomponenttypeid = 3
               AND d.keydeveventtypeid = 48
               AND e.speakertypeid = 3
             ORDER BY b.keydevid,
                      b.transcriptcollectiontypeid,
                      a.transcriptid,
                      a.componentorder
           '''

In [ ]:
snp500_transcripts = db.raw_sql(sql_query)